### Awaitable объекты

In [4]:
import asyncio
import time

class Timer:
    def __init__(self, coro):
        self.coro = coro
        self.spent = 0
    
    def __await__(self):
        started = time.monotonic()
        result = yield from self.coro.__await__()
        self.spent = time.monotonic() - started
        return result
    
async def main():
    t = Timer(asyncio.sleep(1, 'test'))
    res = await t #vibo: ждем завершение Timer, только после этого возвращаем результат
    print(res) # test
    print(t.spent) # 1.00341845

#vibo: для запуска в jupyter
await main()

test
1.002049745999102


#### Что такое coroutine?

In [6]:
import asyncio

async def actual_coro():
    print('hello')
    await asyncio.sleep(3)
    print('world')

#coro = actual_coro()

#vibo: для запуска в jupyter
await actual_coro()

hello
world


In [7]:
import asyncio

@asyncio.coroutine
async def old_coro():
    print('hello')
    yield from asyncio.sleep(1)
    print('world')

#coro = old_coro()


SyntaxError: 'yield from' inside async function (1837114281.py, line 6)

#### Как запустить сопрограмму?

##### asyncio.run()

In [8]:
import asyncio

async def get_text():
    return 'hello, world!'

async def say_text():
    text = await get_text()
    await asyncio.sleep(1)
    return(text)

#result = asyncio.run(say_text())
#print(result) # hello, world!

await say_text()

'hello, world!'

##### asyncio.create_task()

In [9]:
async def get_text(delay, text):
    await asyncio.sleep(delay)
    return text

async def say_text():
    task1 = asyncio.create_task(get_text(5, 'hello'))
    task2 = asyncio.create_task(get_text(5, 'world'))
    await task1
    await task2
    return ', '.join([task1.result(), task2.result()])

#result = asyncio.run(say_text())
#print(result) # hello, world

await say_text()


'hello, world'

#### Что такое asyncio.Task и зачем он нужен?

In [ ]:
import asyncio
import random

rate = random.randint(1, 99)

async def handle(reader, writer):
    writer.write(f'Dollar rate is {rate}. \n'.encode())
    await writer.drain()
    writer.close()

async def main():
    server = await asyncio.start_server(handle, '127.0.0.1', 8888)
    async with server:
        await server.serve_forever()

#asyncio.run(main())

await main()

Для обновления информации допишем update_rate() 

In [ ]:
import asyncio
import random

rate = random.randint(1, 99)

async def update_rate():
    global rate
    while True:
        rate = random.randint(1, 99)
        await asyncio.sleep(10)

async def main():
    asyncio.create_task(update_rate())
    server = await asyncio.start_server(handle, '127.0.0.1', 8888)
    async with server:
        await server.serve_forever()

#asyncio.run(main())
await main()

#### Что такое asyncio.Future?

In [8]:
import asyncio
import time

async def my_sleep(delay):
    loop = asyncio.get_running_loop()
    future = loop.create_future()
    loop.call_later(delay, future.set_result, True)
    await future

async def main():
    loop = asyncio.get_running_loop()
    print(loop.time()) # 0.9779213
    await my_sleep(1)
    print(loop.time()) # 1.979251479

#vibo: для запуска из python
#asyncio.run(main())
#vibo: для запуска в jupyter
await main()

12302.324894362
12303.326730096


### Высокоуровневый API

#### Как приостановить задачу на некоторое время?

In [9]:
import asyncio

async def main():
    print(await asyncio.sleep(1, 'hello'))

#vibo: для запуска из python
#asyncio.run(main())
#vibo: для запуска в jupyter
await main()

hello


#### Как запустить сразу несколько задач?

In [34]:
import aiohttp

async def check_user_exists(user_id: int) -> bool:
    async with aiohttp.ClientSession() as session:
        url = f'https://example.org/users/{user_id}'
        async with session.head(url) as resp:
            print(user_id, resp.status == 200)
            return resp.status == 200

In [35]:
import asyncio

async def main():
    for i in range(10):
        await check_user_exists(i)

#vibo: для запуска из python
#asyncio.run(main())
#vibo: для запуска в jupyter
await main()

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False


Пробуем сделать лучше

In [36]:
async def main():
    future = asyncio.get_running_loop().create_future()
    tasks = []
    executing = 0

    def cb(task):
        nonlocal executing
        executing -= 1
        if executing == 0:
            future.set_result([task.result() for task in tasks])
    
    for i in range(10):
        task = asyncio.create_task(check_user_exists(i))
        task.add_done_callback(cb)
        executing += 1
        tasks.append(task)
    
    await future
    print(future.result())

In [37]:
await main()

6 False
7 False
0 False
4 False
8 False
2 False
5 False
9 False
3 False
1 False
[False, False, False, False, False, False, False, False, False, False]


In [41]:
import asyncio

async def main():
    coros = (
        check_user_exists(i)
        for i in range(10)
    )
    
    # <class 'list'>: [False, False, ...]
    results = await asyncio.gather(*coros)

await main()

1 False
3 False
9 False
6 False
8 False
0 False
4 False
2 False
7 False
5 False


#### А что будет если не ждать gather?

In [29]:
import asyncio

async def main():
    coros = (
        check_user_exists(i)
        for i in range(10)
    )
    
    asyncio.gather(*coros)
    await asyncio.sleep(10)

await main()


7 False
6 False
4 False
2 False
1 False
3 False
5 False
0 False
9 False
8 False


#### Что будет в случае исключения в одной из задач в gather?

In [42]:
async def trigger(position):
    await asyncio.sleep(position)
    if position == 3:
        raise RuntimeError('Boooom!')
    print ('%d is OK' % position)

async def russian_roulette():
    coros = (trigger(i) for i in range(8))
    try:
        await asyncio.gather(*coros)
    except RuntimeError as e:
        print(e)
    await asyncio.sleep(10)

#asyncio.ru(russian_roulette)
await russian_roulette()

0 is OK
1 is OK
2 is OK
Boooom!
4 is OK
5 is OK
6 is OK
7 is OK


#### Как отменить задачу?

In [3]:
async def coro():
    print('start')
    await asyncio.sleep(2)
    print('finished')

async def cancel(task):
    await asyncio.sleep(0.5)
    task.cancel()
    print('task.cancel() called')
    try:
        await task
    except asyncio.CancelledError:
        print('task successfully cancelled')

async def main():
    task = asyncio.create_task(coro())
    asyncio.create_task(cancel(task))
    await asyncio.sleep(5)
    assert task.cancelled()

await main()

start
task.cancel() called
task successfully cancelled


#### Зачем ждать отмененную задачу?

In [1]:
async def check_user_exists(user_id: int) -> bool:
    async with aiohttp.ClientSession() as session:
        try:
            url = f'https://example.org/users/{user_id}'
            async with session.head(url) as resp:
                print(user_id, resp.status == 200)
                return resp.status == 200
        finally: # task.cancel()
            await session.post(
                'http://stat.com/checked-users',
                json={'user_id': user_id}
            )

#### Как защитить задачу от отмены?

In [58]:
async def coro():
    print('start')
    try:
        await asyncio.sleep(2)
    except asyncio.CancelledError:
        print('ninja coroutine!')
    print('finished')

async def main():
    task = asyncio.create_task(coro())
    asyncio.create_task(cancel(task))
    await asyncio.sleep(5)
    assert not task.cancelled()

#asyncio.run(main())
await main()

start
task.cancel() called
ninja coroutine!
finished


#### asyncio.shield() защищает awaitable объект от отмены

In [4]:
import asyncio

async def coro():
    print('start')
    await asyncio.sleep(2)
    print('finished')

async def main():
    task = asyncio.create_task(coro())
    shielded = asyncio.shield(task)
    asyncio.create_task(cancel(shielded))
    await asyncio.sleep(5)
    assert not task.cancelled()

#asyncio.run(main())
await main()

start
task.cancel() called
task successfully cancelled
finished


#### Что происходит с ожидающим отмененной задачи, которую отменили?

In [60]:
async def coro():
    print('start')
    await asyncio.sleep(2)
    print('finished')

async def main():
    task = asyncio.create_task(coro())
    shielded = asyncio.shield(task)
    asyncio.create_task(cancel(shielded))
    await shielded
    assert not task.cancelled()

#asyncio.run(main())
await main()


start
task.cancel() called
task successfully cancelled


CancelledError: 

#### Как ограничить время ожидания awaitable объекта (таймаут)?

In [5]:
async def eternity():
    try:
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print('I was cancelled')
        raise
    print('finished')

async def main():
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout')

#asyncio.run(main())
await main()

I was cancelled
timeout


#### Как подождать выполнения awaitable объектов?

In [3]:
import asyncio
import aiohttp

async def main():
    tasks = [
        asyncio.create_task(check_user_exists(i))
        for i in range(10)
    ]

    done, pending = await asyncio.wait(tasks, timeout=1)
    # done: <class 'set'>: {<Task finished coro=...>}
    # pending: <class 'set'>: {<Task finished coro=...>}
    print(done, pending)

#asyncio.run(main())
await main()

5 False
3 False
7 False
4 False
8 False
6 False
0 False
9 False
1 False
2 False
set() {<Task pending name='Task-7' coro=<check_user_exists() running at /tmp/ipykernel_244/1346784782.py:9> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fc763856d60>()]>>, <Task pending name='Task-12' coro=<check_user_exists() running at /tmp/ipykernel_244/1346784782.py:9> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fc7638e2370>()]>>, <Task pending name='Task-14' coro=<check_user_exists() running at /tmp/ipykernel_244/1346784782.py:9> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fc763873940>()]>>, <Task pending name='Task-10' coro=<check_user_exists() running at /tmp/ipykernel_244/1346784782.py:9> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fc763856d30>()]>>, <Task pending name='Task-8' coro=<check_user_exists() running at /tmp/ipykernel_244/1346784782.py:9> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fc76

In [4]:
import asyncio
import aiohttp

async def main():
    tasks = [
        asyncio.create_task(check_user_exists(i))
        for i in range(10)
    ]

    done, pending = await asyncio.wait(
        tasks, return_when=asyncio.FIRST_EXCEPTION
    )
    # done: <class 'set'>: {<Task finished rn=...>}
    # pending: <class 'set'>: {<Task finished coro=...>}
    print(done, pending)

#asyncio.run(main())
await main()

7 False
1 False
4 False
2 False
8 False
0 False
3 False
5 False
9 False
6 False
{<Task finished name='Task-23' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-25' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-19' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-21' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-17' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-24' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-26' coro=<check_user_exists() done, defined at /tmp/ipykernel_245/1346784782.py:1> result=False>, <Task finished name='Task-22' coro=<check_user_exis

#### Как еще подождать выполнения awaitable объектов?

In [2]:
import asyncio
import aiohttp

async def dalayed_result(delay) -> bool:
    return await asyncio.sleep(delay, delay)

async def main():
    tasks = [
        dalayed_result(i)
        for i in range(10)
    ]

    for earliest in asyncio.as_completed(tasks):
        result = await earliest
        print(result)

#asyncio.run(main())
await main()


0
1
2
3
4
5
6
7
8
9


#### Что сейчас выполняется?

In [1]:
import asyncio

async def main():
    task = asyncio.current_task()
    # <Task pending coro=<main() running at ...>
    print(task)

#asyncio.run(main())
await main()

<Task pending name='Task-4' coro=<InteractiveShell.run_cell_async() running at /home/vibo/vs_code/venv-vsc/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3135> cb=[IPythonKernel._cancel_on_sigint.<locals>.cancel_unless_done(<Future pendi...ernel.py:271]>)() at /home/vibo/vs_code/venv-vsc/lib/python3.9/site-packages/ipykernel/ipkernel.py:271, <TaskWakeupMethWrapper object at 0x7f2b180e5c70>()]>


In [ ]:
import asyncio
import aiohttp

async def main():
    coros = (
        check_user_exists(i)
        for i in range(10)
    )
    asyncio.gather(*coros)
    await asyncio.sleep(0.1)
    tasks = asyncio.all_tasks()
    print(type(tasks))
    print(tasks)

#asyncio.run(main())
await main()

### Низкоуровневый API

#### Что скрывается за uvloop.install()?

In [ ]:
def install():
    """A helper function to install uvloop policy."""
    __asyncio.set_event_loop_policy(EventLoopPolicy())

class EventLoopPolocy(__BasePolicy):
    """Event loop policy.
    The preferred way to make your application use uvloop:
    >>> import asyncio
    >>> import uvloop
    >>>
asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())
    >>> asyncio.get_event_loop(
    <uvloop.Loop running=False closed=False debug=False>
    """

#### Запуск синхронного кода в процессе/потоке

In [ ]:
import asyncio

def blocking_io():
    with open('dev/urandom', 'rb') as f:
        return f.read(100)

def cpu_bound():
    return sum(i * i for i in range(10 ** 7))

async def main():
    loop = asyncio.get_running_loop()
    # 1. Run in the default loop's executor:
    result = await loop.run_in_executor(None, blocking_io)
    print('default thread pool', result)

#asyncio.run(main()
await main()

In [ ]:
async def main():
    loop = asyncio.get_running_loop()

    # 2. Run in a custom thread pool:
    with concurrent.futures.ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, blocking_io)
        print('custom thread pool', result)
    
    #3. Run in a custom process pool:
    with concurrent.futures.ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, cpu_bound)
        print('custom process pool', result)

#asyncio.run(main())
await main()

### Асинхронные интерфейсы

#### Как реализовать асинхронный менеджер контекста?

In [10]:
class TransactionCtx:
    def __init__(self, conn):
        self.conn = conn
    
    async def __aenter__(self):
        await self.conn.execute('BEGIN')
        print('entering context')
        return self
    
    async def __aexit__(self, exc_type, exc, tb):
        command = 'ROLLBACK' if exc else 'COMMIT'
        await self.conn.execute(command)
        print('exiting context')

#### Как написать асинхронный итератор?

In [11]:
class Ticker:
    def __init__(self, delay, to):
        self.delay = delay
        self.i = 0
        self.to = to
    
    def __aiter__(self):
        return self

    async def __anext__(self):
        i = self.i
        if i >= self.to:
            raise StopAsyncIteration
        self.i += 1
        if i:
            await asyncio.sleep(self.delay)
        return i

#### Как использовать асинхронный итератор?

In [12]:
async def main():
    async for i in Ticker(1, 10):
        print(i)
    else:
        print('hm?')

#asyncio.run(main())
await main()

0
1
2
3
4
5
6
7
8
9
hm?


#### Как реализовать и использовать асинхронный генератор?

In [14]:
async def ticker(delay, to):
    for i in range(to):
        yield i
        await asyncio.sleep(delay)

async def main():
    async for i in ticker(1, 10):
        print(i)

#asyncio.run(main())
await main()

0
1
2
3
4
5
6
7
8
9


#### Asynchronous comprehensions

In [16]:
async def ticker(delay, to):
    for i in range(to):
        yield i
        await asyncio.sleep(delay)
    
async def main():
    results = [
        (i, j)
        async for i in ticker(0.1, 5)
        async for j in ticker(0.1, 5)
        if not i % 2 and j % 2
    ]
    print(results)

await main()

[(0, 1), (0, 3), (2, 1), (2, 3), (4, 1), (4, 3)]


### Aiohttp

#### Что это и зачем?

In [29]:
import asyncio

async def handle(reader, writer):
    # get client headers
    # parse request headers
    # route request (call appropriate handler)
    # retrieve request body
    # write response headers
    # write response payload (body)
    pass

async def main():
    server = await asyncio.start_server(handle, '127.0.0.1', 8888)
    async with server:
        await server.serve_forever()

#asyncio.run(main())
await main()

#### Пример HTTP сервера

In [ ]:
from aiohttp import web
from aiohttp.web import run_app

async def handle(request):
    return web.Response(text='Meow.')

app = web.Application()
app.router.add_route('GET', '/', handle)
run_app(app, port=8888)

#### Пример HTTP клиента

In [ ]:
import asyncio
import aiohttp

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get('http://0.0.0.0:8888') as resp:
            print(resp.status) # body не можем распечатать - payloads
            print(await resp.text)

asyncio.run(main())

### Middleware

#### Что такое middleware

In [ ]:
async def test(request):
    print('Handler function called')
    return web.Response(text='Hello')

@web.middleware
async def middleware(request, handler):
    print('Middleware called')
    response = await handler(request)
    print('Middleware finished')
    return response

app = web.Application(middlewares=[middleware])
app.router.add_get('/', test)
web.run_app(app)

### Payloads

#### Напишем простое aiohttp приложение

In [ ]:
import asyncpg

async def init_pg(app):
    app['pg'] = await asyncpg.create_pool(
        'postgresql://user:me@0.0.0.0/test'
    )

async def handle(request):
    async with request.app['pg'].acquire() as conn:
        row = await conn.fetchrow('SELECT 1 as col')
    return aiohttp.web.Response(body={'data': row})

app = web.Application()
app.router.add_route('GET', '/', handle)
app.on_startup.append(init_pg)
aiohttp.web.run_app(app, port=8082)

#### Что такое JsonPayload?

In [ ]:
from json import JSONEncoder

class JsonPayload(BytesPayload):

    def __init__(self,
                value: Any,
                encoding: str='utf-8',
                content_type: str='application.json',
                dumps: JSONEncoder=json.dumps,
                *args: Any,
                **kwargs: Any) -> None:
        
        super().__init__(
            dumps(value).encode(encoding),
            content_type=content_type, encoding=encoding,
            *args, **kwargs)

#### singledispatch for win

In [2]:
from datetime import datetime
from functools import singledispatch
from asyncpg import Record

@singledispatch
def convert(value):
    raise NotImplementedError(f'Unserializable value: {value!r}')

@convert.register(Record)
def convert_asyncpg_record(value: Record):
    return dict(value)

@convert.register(datetime)
def convert_datetime(value: datetime):
    return value.isoformat()

####  Научим aiohttp работать с объектами типа asyncpg.Record и datetime

In [ ]:
from email.policy import default
import json
from functools import partialmethod, partial
from typing import Mapping

dumps = partial(json.dumps, default=convert)

class UniversalJsonPayload(JsonPayload):
    __init__ = partialmethod(JsonPayload.__init__, dumps=dumps)

PAYLOAD_REGISTRY.register(UniversalJsonPayload, Mapping)

### Проблемы



#### Управление фоновыми задачами



##### Запуск и остановка Event Loop

In [ ]:
import asyncio, time

async def handler(reader, writer):
    writer.write(f'{time.time()}\n\r'.encode())
    writer.close()

async def server():
    server = await asyncio.start_server(handler, '::1', 2023)
    await server.serve_forever

async def main():
    await server()

#asyncio.run(main())
await main()


##### Упаавление задачами

In [ ]:
async def cache(interval):
    """ Start updateing response cache """
    pass

async def server():
    """ Start TCP server """
    pass

async def statistic():
    """ Start sending usage statistic """
    pass

async def log_sender():
    """ Start sending internal logs to LOG collection """
    pass

async def main():
    await asyncio.gather(cache(), server(), statistic(), log_sender())

asyncio.run(main())

##### Прототипы задач

In [ ]:
async def cache(interval):
    """ Start updateing response cache """
    while True:
        await update_global_cache()
        await asyncio.sleep(interval)

async def server():
    """ Start TCP server """
    await passing_config_to_handler()
    await create_server()

async def statistic():
    """ Start sending usage statistic """
    await configuring_sender()
    await start_sending()

async def log_sender():
    """ Start sending internal logs to LOG collection """
    await configuring_sender()
    await start_sending()

async def main():
    await asyncio.gather(cache(), server(), statistic(), log_sender())

asyncio.run(main())

##### Решение с wait

In [ ]:
async def main():
    done, pending = await asyncio.wait(
        [
            cache(),
            server(),
            statistic(),
            log_sender(),
        ],
        return_when=asyncio.FIRST_EXCEPTION
    )
    pass

asyncio.run(main())

#### Stdout блокируется

##### Простой пример

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

for i in range(1000):
    logging.info("Iteration %s", i)

##### Простой пример (умный сервис)

In [ ]:
import aiohttp, asyncio

async def fetch(session, url):
    while True:
        try:
            async with session.get(url) as response:
                return await response.text()
        except aiohttp.ClientError:
            await asyncio.sleep(0.5)

async def main():
    async with aiohttp.ClientSession() as session:
        print(await fetch(session, 'http://python.org'))

#asyncio.run(main())
await main()

##### ThreadPool/ProcessPool

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

loop = asyncio.new_event_loop()
loop.set_default_executor(ThreadPoolExecutor(8))

def reader(fname):
    loop = asyncio.get_running_loop()
    def read_file():
        with open(fname, "rb") as fp:
            return fp.read()
    return loop.run_in_executor(None, read_file)

async def main():
    print(await reader("/ect/bashrc"))

loop.run_until_complete(main())

### Основные концепции



#### Entrypoint


##### Запуск и остановка EventLo

In [3]:
!pip install aiomisc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 626.4 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for logging-journald, since package 'wheel' is not installed.
  Running setup.py install for logging-journald ... done


In [ ]:
import asyncio
import aiomisc

async def main():
    await asyncio.sleep(1)

with aiomisc.entrypoint() as loop:
    loop.run_until_complete(main()) # analog asyncio.run()


In [ ]:
import asyncio, time, aiomisc

async def hanler(reader, writer):
    writer.write(f"{time.time()}\n\r".encode())
    writer.close()

async def server():
    server = await asyncio.start_server(handler, "::1", 2023)

async def main():
    await server()

with aiomisc.entrypoint() as loop:
    loop.create_task(main())
    loop.run_forever()

##### Что еще умеет делать entrypoint?

In [ ]:
async def main():
    while True:
        await asyncio.sleep(1)
        logging.info('Hello there')

with aiomisc.entrypoint(
    pool_size=2,            # thread pool size
    log_level='info',       # logging configuratiion
    log_format='color',     # log formatter
    log_buffer_size=1024,   # buffer size for log records
    log_flush_interval=0.2, # write logs in separate thread
    debug=False,            # set debug flag for event loop
    log_config=True,
    policy=asyncio.DefaultEventLoopPolicy(),
) as loop:
    loop.create_task(main())
    loop.run_forever()

### Сервисы



#### Сервис - базовый класс

In [ ]:
import asyncio
from aiomisc import entrypoint, Service

class MyService(Service):
    async def start(self):
        self.task = self.loop.create_task(
            asyncio.sleep(3600, loop=self.loop)
        )
    
    async def stop(self):
        self.task.cancel()

        try:
            await self.task
        except asyncio.CancelledError:
            pass

In [ ]:
import asyncio
from aiomisc import entrypoint, Service

class MyService(Service):
    async def start(self):
        self.start_event.set()
        await asyncio.sleep(3600, loop=self.loop)
    

#### Как запустить сервис?

In [ ]:
with entrypoint(MyService()) as loop:
    loop.run_forever()

#### Как запустить сервисы?

In [ ]:
import asyncio
from aiomisc import entrypoint, Service

class MyService(Service):
    async def start(self):
        self.start_event.set()
        await asyncio.sleep(3600, loop=self.loop)

s1 = MyService()
s2 = MyService()

with entrypoint(s1, s2, log_level='info') as loop:
    loop.run_forever()

#### Базовые классы сервисов



##### TCPServer

In [ ]:
from aiomisc.entrypoint import entrypoint
from aiomisc.service import TCPServer

class EchoServer(TCPServer):
    async def handle_client(self, reader, writer):
        while True:
            writer.write(await reader.readline())

echo = EchoServer(address='::1', port=8901)

with entrypoint(echo) as loop:
    loop.run_forever()

##### TLSServer

In [ ]:
from aiomisc.entrypoint import entrypoint
from aiomisc.service import TLSServer

class EchoServer(TLSServer):
    async def handle_client(self, reader, writer):
        while True:
            writer.write(await reader.readline())

ssl_opts = dict(ca='ca.pem', cert='cert.pem', key='key.pem')
echo = SecureEchoServer(address='::1', port=8900, **ssl_opts)

with entrypoint(echo) as loop:
    loop.run_forever()

##### UDPServer

In [ ]:
from aiomisc.entrypoint import entrypoint
from aiomisc.service import UDPServer

class UDPPrinter(UDPServer):
    async def handle_dategram(self, data: bytes, addr):
        print(addr, '->', data)

service = UDPPrinter(address='::1', port=3000)

with entrypoint(service) as loop:
    loop.run_forever()

##### PeriodicService

In [ ]:
import aiomisc
from aiomisc.service.periodic import PeriodicService

class MyPeriodicService(PeriodicService):
    async def callback(self):
        log.info('Running periodic callback')
        #...

service = MyPeriodicService(interval=3600) # once per hour

with entrypoint(service) as loop:
    loop.run_forever()

##### Кофигурирование сервисов

In [ ]:
class LoggingService(Service):
    __required__ = frozenset({'name'}) # required kwargs

    delay: int = 1                     # default value

    async def start(self):
        self.start_event.set()
        while True:
            print('Hello from service', self.name)
            await asyncio.sleep(self.delay)

services = (
    LoggingService(name='#1'),
    LoggingService(name='#2', delay=3),
)

with entrypoint(*services) as loop:
    loop.run_forever()

##### aiohttp service

In [ ]:
async def handle(request):
    name = request.match_info.get('name', "Anonymous")
    return aiohttp.web.Response(text='Hello, ' + name)

class REST(AIOHTTPService):
    async def create_application(self):
        app = aiohttp.web.Application()

        app.add_routes([
            aiohttp.web.get('/', handle),
            aiohttp.web.get('/{name}', handle)
        ])

        return app
    
with entrypoint(REST(address="::", port=8081)) as loop:
    loop.run_forever()

### Готовые сервисы



#### MemoryTracer

In [ ]:
import asyncio
import os
from aiomisc import entrypoint
from aiomisc.service import MemoryTracer

async def main():
    leaking = []

    while True:
        leaking.append(os.urandom(128))
        await asyncio.sleep(0)

with entrypoint(MemoryTracer(interval=1, top_results=5)) as loop:
    loop.run_until_complete(main())

#### Profiler

In [ ]:
import asyncio
import os
from aiomisc import entrypoint
from aiomisc.service import Profiler

async def main():
    for i in range(100):
        time.sleep(0.01)

with entrypoint(Profiler(interval=0.1, top_results=5)) as loop:
    loop.run_until_complete(main())

### Декораторы

#### @timeout

In [8]:
from aiomisc import timeout

@timeout(1)
async def bad_func():
    await asyncio.sleep(2)

#### @asyncbackoff

In [11]:
from typing import Type
from aiomisc import asyncbackoff

attempt_timeout = 0.1
deadline = 1
pause = 0.1

@asyncbackoff(attempt_timeout, deadline, pause)
async def db_fetch():
    pass

# Passing exceptions for handling
@asyncbackoff(0.1, 1, 0.1, TypeError, RuntimeError, ValueError)

async def db_fetch(data: dict):
    pass

In [ ]:
# Will be retried no more then 2 times (3 tries total)
@asyncbackoff(attempt_timeout=0.5, 
                deadline=1,
                pause=0.1,
                max_tries=3,
                exceptions=[TypeError, RuntimeError, ValueError])
async def db_fetch(data: dict):
    pass

# Will be retried only on connection abort (on POSIX systems)
@asyncbackoff(attempt_timeout=0.5, 
                deadline=1,
                pause=0.1,
                exceptions=[OSError],
                giveup=lambda e: e.errno != errno.ECONNABORTED)
async def db_fetch(data: dict):
    pass

#### FileIO

In [ ]:
from aiomisc.io import async_open

async def file_write():
    async with async_open('/tmp/test', 'w+') as afp:
        await afp.write("Hello")
        await afp.write(" ")
        await afp.write("world")

        await afp.seek(0)
        print(await afp.read())

### Работа с потоками

#### @aiomisc.threaded

In [ ]:
import asyncio, time, aiomisc

@aiomisc.threaded
def blocking_function():
    time.sleep(1)

async def main():
    # Running in parallel
    await asyncio.gather(
        blocking_function(),
        blocking_function(),
    )

with aiomisc.entrypoint() as loop:
    loop.run_until_complete(main())